In [45]:
import pandas as pd
import numpy as np
from copy import deepcopy
import time
from joblib import Parallel, delayed
import multiprocessing
import gc

In [46]:
t0=time.time()

In [2]:
pd.set_option('max_rows',2000)
pd.set_option('max_columns',2000)
np.set_printoptions()

In [3]:
#df = pd.read_stata('/scratch/sv1239/projects/mlcs/raw/Votelevel_stuffjan2013.dta')
df = pd.read_stata('../raw/Votelevel_stuffjan2013.dta')

In [4]:
df.shape

(56403, 868)

In [5]:
# columns to be removed, casenum or codej1... or j2vote... or j2maj... will be removed later
del_cols = ['fileid','cite','vol','beginpg','endopin','endpage','docnum','priorpub','_merge','year',
            'circuit','pseatno','decision_date','aatty_first_name','aatty_last_name','afirm_name',
            'ratty_first_name','ratty_last_name','rname_of_first_listed_amicus_gro','rfirm_namew','decisiondatenew2',
           'j1name','j2name','j3name','quartertoelect','pname','seatno','success','lsuc','ls1','ls2','ls3','lp',
            'lp2','lp3','sseatno','congress','congreso','afirst_listed_amicus_group','yearquarter','name','Name','State','j',
            'codej4','j4vote1','j4vote2','j4maj1','j4maj2','codej5','j5vote1','j5vote2','j5maj1','j5maj2',
            'codej6','j6vote1','j6vote2','j6maj1','j6maj2','codej7','j7vote1','j7vote2','j7maj1','j7maj2',
            'codej8','j8vote1','j8vote2','j8maj1','j8maj2','codej9','j9vote1','j9vote2','j9maj1','j9maj2',
            'codej10','j10vote1','j10vote2','j10maj1','j10maj2','codej11','j11vote1','j11vote2','j11maj1','j11maj2',
            'codej12','j12vote1','j12vote2','j12maj1','j12maj2','codej13','j13vote1','j13vote2','j13maj1','j13maj2',
            'codej14','j14vote1','j14vote2','j14maj1','j14maj2','codej15','j15vote1','j15vote2','j15maj1','j15maj2','j16maj1','j16vote1']
#
# to_dummies = ['month','day','method','state','district','origin','source','distjudg',
#               'applfrom','adminrev','opinstat','treat','classact','crossapp','counsel1','counsel2','sanction',
#               'initiate','numappel','appnatpr','appnatpr','appbus','appnonp','appfed','appsubst','appstate',
#               'appfiduc','ap_stid','genapel1','bank_ap1','genapel2','bank_ap2','appel1','appel2',]

In [6]:
df.head()

,casenum,fileid,year,month,day,cite,vol,beginpg,endopin,endpage,docnum,multdoc,method,circuit,state,district,origin,source,distjudg,applfrom,adminrev,priorpub,opinstat,treat,classact,crossapp,majvotes,dissent,concur,counsel1,counsel2,amicus,sanction,initiate,interven,numappel,appnatpr,appbus,appnonp,appfed,appsubst,appstate,appfiduc,ap_stid,genapel1,bank_ap1,genapel2,bank_ap2,appel1,appel2,realapp,numresp,r_natpr,r_bus,r_nonp,r_fed,r_subst,r_state,r_fiduc,r_stid,respond1,respond2,realresp,genresp1,genresp2,bank_r1,bank_r2,const1,const2,usc1,usc1sect,usc2,usc2sect,civproc1,civproc2,crmproc1,crmproc2,juris,statecl,standing,mootness,exhaust,timely,immunity,frivol,polquest,oththres,late,frivapp,othappth,constit,fedlaw,procedur,typeiss,prejud,insane,improper,juryinst,othjury,deathpen,sentence,indict,confess,search,othadmis,plea,counsel,rtcouns,suffic,indigent,entrap,procdis,othcrim,dueproc,execord,stpolicy,weightev,pretrial,trialpro,post_trl,attyfee,judgdisc,altdisp,injunct,summary,fedvst,foreign,int_law,st_v_st,discover,othcivil,subevid,denovo,erron,capric,abusedis,judrev,genstand,notice,alj,agen_acq,freeinfo,comment,record,diverse,whlaws,casetyp1,geniss,direct1,casetyp2,direct2,habeas,decuncon,unan,signed,ids,codej2,j2vote1,j2vote2,j2maj1,j2maj2,codej3,j3vote1,j3vote2,j3maj1,j3maj2,codej4,j4vote1,j4vote2,j4maj1,j4maj2,codej5,j5vote1,j5vote2,j5maj1,j5maj2,codej6,j6vote1,j6vote2,j6maj1,j6maj2,codej7,j7vote1,j7vote2,j7maj1,j7maj2,codej8,j8vote1,j8vote2,j8maj1,j8maj2,codej9,j9vote1,j9vote2,j9maj1,j9maj2,codej10,j10vote1,j10vote2,j10maj1,j10maj2,codej11,j11vote1,j11vote2,j11maj1,j11maj2,codej12,j12vote1,j12vote2,j12maj1,j12maj2,codej13,j13vote1,j13vote2,j13maj1,j13maj2,codej14,j14vote1,j14vote2,j14maj1,j14maj2,codej15,j15vote1,j15vote2,j15maj1,j15maj2,pagelgth,j16maj1,j16vote1,citevol,decision_date,alst_code,nofirmap,noattyap,aatty_first_name,aatty_last_name,afirm_name,amicusapp,a_groups,afirst_listed_amicus_group,r_appsubst,rlst_code,nofirmrp,noattyrp,ratty_first_name,ratty_last_name,amicusresp,r_groups,rname_of_first_listed_amicus_gro,isspty,dissmer,concmer1,concmer2,disspro,concpro1,concpro2,fedlaw2,keytotal,threshgen,turnonthresh,rfirm_namew,decisiondatenew2,dropflag,j1name,j1party,j1appres,j1distjudge,j1sample,j1score,moved,j2name,j2party,j2appres,j2distjudge,j2sample,j2score,j3name,j3party,j3appres,j3distjudge,j3sample,j3score,sumappress,divided,oldj1party,oldj2party,oldj3party,sumparty,divided1,sumpartyappt,dividedA,districtjudgeinpanel,nomoversinpanel,concurdummy,dissentdummy,unanimous,quarter,fromdistrictcourt,typeiss2,geniss2,geniss3,yearquarter,decade2,decade,distance,quartertoelect,semiannumtoelect,lastquarter,sixmonthsbefore,sixmonthsafter,lengthopin,Wopinionlenght,totalcites,negativecites,propneg,Wtotalcites,Wlengthopin,lastthreequarter,dissmerdummy,dissprodummy,disspromerdummy,disspro0merdummy,diss0promerdummy,diss0pro0merdummy,recent,type1,type2,type3,type4,type5,type6,type7,affirmdummy,reversedummy,initiate1,initiate2,initiate3,initiate4,initiate5,concmerdummy,concprodummy,concpromerdummy,concpro0merdummy,conc0promerdummy,conc0pro0merdummy,month911_f4,month911_f3,month911_f2,month911_f1,month911_t0,month911_t1,month911_t2,month911_t3,month911_t4,month911_t5,month911_t6,month911_3mon,last3,ww2,korean,vietnam,gulf,afghan,war,clinton,caseload,president,senate,president_f1,president_f2,senate_f1,senate_f2,electoralpct,popularpct,var10,var11,repres,rsen,repres_f1,rsen_f1,repres_f2,rsen_f2,j,codej1,dissentvote,concurauthor,concurjoin,pname,id,pres,mona,yeara,monl,yearl,yearb,yeard,pleft,left,party,city,elevate,gender,race,name,appres,pleftsum,boom,crossa,crossl,mrespty,prespty,dummy,religion,csb,ba,bast,bapp,ls,lsst,jdpp,graddeg1,graddeg2,graddeg3,pfedjdge,pfjdget,pssc,pslc,plocct,poutsjge,psjdget,pausa,pusa,psgo,psg,pago,paag,pag,pcc,psp,pspros,pmag,pbj,pcab,pcabdept,pscab,pscabdpt,pindreg1,preg1,pindreg2,preg2,pindreg3,preg3,pindreg4,preg4,pindreg5,preg5,phouse,psenate,pgov,pssenate,pshouse,pmayor,pccoun,pccom,pda,plother

In [7]:
df.drop(labels=del_cols,axis=1,inplace=True)

In [8]:
moredropcolumns=df.columns.tolist() # .tolist?
for i in moredropcolumns:
    if len(pd.unique(df[i]))==1:
        df.drop(labels=i,axis=1,inplace=True)

In [11]:
caseList=pd.unique(df['casenum'])
caseList=caseList[pd.notnull(caseList)].tolist()
print len(caseList)

18585


In [14]:
num_cores = multiprocessing.cpu_count()

In [48]:
print "num_cores is: ",num_cores

num_cores is:  20


In [18]:
def do_to_case(case):
    newframe=pd.DataFrame()                ##  the rearrange of the original data
    output=[]                           ##   the corresponding alignment of judge 1 and judge 2, yes =1, no = -1    
    subtest=df[df.casenum==case].reset_index(drop=True)  ## 'subtest' only take the records that have a specific case id
    num=subtest.shape[0]                                 ## num will be 3, because usally there are 3 records for each case 
    for j in range(num):
        if pd.notnull(subtest.ix[j].codej1):
            j1=subtest.ix[j].codej1
        if pd.notnull(subtest.ix[j].codej2):
            j2=subtest.ix[j].codej2
        if pd.notnull(subtest.ix[j].codej3):
            j3=subtest.ix[j].codej3
    if subtest.ix[0].majvotes == 3:
        j1j2=1
        j1j3=1
        j2j3=1
    else:
        if subtest.ix[0].direct1==subtest.ix[0].j2vote1:
            j1j2=1
        else:
            j1j2=-1                
        if subtest.ix[0].direct1==subtest.ix[0].j3vote1:
            j1j3=1
        else:
            j1j3=-1
        if subtest.ix[0].j2vote1==subtest.ix[0].j3vote1:
            j2j3=1
        else:
            j2j3=-1
    for j in range(num):
        copytest=deepcopy(subtest.ix[j])
        if copytest.ids==j1:
            copytest.codej1=j1
            newframe=newframe.append(copytest)
            output.append(j1j2)
            copytest.codej2=j3
            copytest.codej3=j2
            newframe=newframe.append(copytest)
            output.append(j1j3)
        if copytest.ids==j2:
            copytest.codej1=j2
            copytest.codej2=j1
            newframe=newframe.append(copytest)
            output.append(j1j2)
            copytest.codej2=j3
            copytest.codej3=j1
            newframe=newframe.append(copytest)
            output.append(j2j3)
        if copytest.ids==j3:
            copytest.codej1=j3
            copytest.codej3=j1
            newframe=newframe.append(copytest)
            output.append(j2j3)
            copytest.codej2=j1
            copytest.codej3=j2
            newframe=newframe.append(copytest)
            output.append(j1j3)

#     newframe=newframe.reset_index()              ## need to reset the index, otherwise will all be 0
#     newframe=newframe.drop('index',1)  

    assert newframe.shape[0]==len(output)
    print newframe.shape
    return newframe, output

In [23]:
a=Parallel(n_jobs=num_cores)(delayed(do_to_case)(case) for case in caseList[:10])
bigout=[]
bignew=pd.DataFrame()
for x in a:
    if bignew.empty:
        bignew=x[0]
        print "empty"
    else:
        bignew=pd.concat([bignew,x[0]],ignore_index=True)
    bigout = bigout + x[1]
    gc.collect()

empty
CPU times: user 300 ms, sys: 433 ms, total: 733 ms(6, 743)
(6, 743)
(6, 743)
(6, 743)
(6, 743)
(6, 743)
(6, 743)
(6, 743)
(6, 743)
(6, 743)

Wall time: 1.94 s


In [24]:
# %%time
# bigout2=[]
# bignew2=pd.DataFrame()
# for case in caseList[:10]:
#     a,b=do_to_case(caseList[0])
#     if bignew2.empty:
#         bignew2=a
#         print "empty"
#     else:
#         bignew2=pd.concat([bignew2,a],ignore_index=True)
#     bigout2 = bigout2 + b
#     gc.collect()

(6, 743)
empty
(6, 743)
(6, 743)
(6, 743)
(6, 743)
(6, 743)
(6, 743)
(6, 743)
(6, 743)
(6, 743)
CPU times: user 3.93 s, sys: 3 ms, total: 3.93 s
Wall time: 3.93 s


In [26]:
## remove the columns that we are done with
bignew.drop(labels=['casenum','j2vote1','j2vote2','j2maj1','direct1',
                      'j2maj2','j3vote1','j3vote2','j3maj1','j3maj2','majvotes','ids'],axis=1,inplace=True)

In [27]:
new_cols=bignew.columns

In [28]:
new_cols=new_cols.tolist()

In [29]:
keep_cols=['j1score','j2score','j3score','popularpct','electoralpct','closerd','fartherd','dAds3','dF2Ads3',
           'dF1Ads3','dL1Ads3','dL2Ads3','dL3Ads3','dL4Ads3','dL5Ads3','logAds3','logL1Ads3','logL2Ads3','logF1Ads3',
          'logF2Ads3','decade2','propneg','likely_elev2','score','d12','d13','d23']

'''
This keep_cols are all float no categories. So null=0 will be fine.

for col in keep_cols:
    if len(pd.unique(newframe[col]))<4:
        print pd.unique(newframe[col]),col+"\n"
'''
#doubt that sth like dl4ads3 is also categories

'\nThis keep_cols are all float no categories. So null=0 will be fine.\n\nfor col in keep_cols:\n    if len(pd.unique(newframe[col]))<4:\n        print pd.unique(newframe[col]),col+"\n"\n'

In [30]:
for col in keep_cols:
    if col in new_cols:
        new_cols.remove(col)

In [33]:
bignew=pd.get_dummies(bignew,columns=new_cols,dummy_na=True)

TypeError: get_dummies() got an unexpected keyword argument 'columns'

In [61]:
bignew=bignew.fillna(value=0)         

In [63]:
bignew.to_csv('/scratch/sv1239/projects/mlcs/mlcs16/out2.csv')
(pd.DataFrame(bigout)).to_csv('/scratch/sv1239/projects/mlcs/mlcs16/output2.csv')

In [43]:
t1=time.time()

In [53]:
print "time in sec:",t1-t0

time in sec: -24.477668047
